In [20]:
import sys
import wget
import newspaper
from newspaper import Article
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import itertools
import lxml
import xmltodict
import collections
from urllib import request
from collections import OrderedDict
from urllib.error import HTTPError, URLError
from urllib.parse import urlparse
import sys
import ast
import time
from pandarallel import pandarallel
import requests
import datefinder
from datetime import datetime
from tqdm import tqdm
import psutil
from selenium import webdriver
from multiprocessing import Pool
import random

In [21]:
country = "mexico"
news_outlet = "diariodexalapa"
csv_name = country+'_'+news_outlet+'.csv'
extract_text_sleep = random.randint(2,4)

In [22]:
def parallelize_dataframe(df, func, n_cores):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [23]:
def extract_text(url):
    text, title, date = None, None, None
    try:
        result = [country, news_outlet] + [''] * 3 
        article = Article(url, keep_article_html=False)
        article.download()
        time.sleep(extract_text_sleep)
        article.parse()
        text = article.text
        
        if text:
            text_copy = text
            title = article.title
            date = article.publish_date
            if date:
                date = date.strftime(date_format)
            else:
                matches = datefinder.find_dates(text_copy)
                most_recent_datetime = sorted(matches)[-1]
                date = most_recent_datetime.strftime(date_format)
    finally:
      if title:
        result[2] = title
      if date:
        result[3] = date
      if text:
        result[4] = text
      return result 


def extract_text_from_df(df):
    content = df['link'].map(lambda x: extract_text(x))
    df['content'] = content
    return df

In [24]:
df = pd.read_csv('links_'+csv_name)
    

In [25]:
df

,Unnamed: 0,link
0,0,https://www.diariodexalapa.com.mx/gossip/diego-del-rio-el-encargado-de-preparar-el-homenaje-a-silvia-pinal-en-bellas-artes-8634032.html
1,1,https://www.diariodexalapa.com.mx/policiaca/asesinan-a-joven-de-tres-disparos...-y-no-hay-detenidos-todo-sobre-el-caso-8633886.html
2,2,https://www.diariodexalapa.com.mx/policiaca/vuelca-camioneta-en-carretera-hacia-perote-que-transportaba-8613069.html
3,3,https://www.diariodexalapa.com.mx/policiaca/secuestradores-de-xalapa-y-coatepec-pasaran-50-anos-de-prision-cuando-ocurrieron-los-delitos-8610004.html
4,4,https://www.diariodexalapa.com.mx/policiaca/presunto-policia-de-la-ssp-extorsionaba-a-duenos-de-bares-en-xalapa-todo-sobre-el-caso-8625181.html
...,...,...
251,251,https://www.diariodexalapa.com.mx/gossip/quien-era-meche-carreno-y-cual-su-legado-en-el-cine-mexicano-8632844.html
252,252,https://www.diariodexalapa.com.mx/?pollResult=7527541#poll1017527541
253,253,https://www.diariodexalapa.com.mx/finanzas/tecnologia/
254,254,https://www.diariodexalapa.com.mx/mundo/china-amaga-con-represalias-contra-eu-ante-posible-visita-de-nancy-pelosi-a-taiwan-8613578.html


In [26]:
output_path= 'unprocessed_' + csv_name
start = 0
limit = 180

while start < len(df):
    start_time = time.time()
    demo_df = df[start:start+limit].copy()
    test_df = parallelize_dataframe(demo_df, extract_text_from_df, 90)
    test_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
    
    test_df['text_set'] = test_df['content'].map(lambda x: True if x[4] != '' else False)    
    no_of_text_retrieved = test_df.text_set.sum()
    print(f'{no_of_text_retrieved} / {len(test_df)} text retrieved')
    
    end_time = time.time()
    print(f'Batch of data of row range {start}-{start+limit} complete in {round(end_time-start_time, 2)} seconds')
    print(f'{round(min((((start+limit) / len(df)) * 100), 100), 2)}% complete')
    start+=limit


176 / 180 text retrieved
Batch of data of row range 0-180 complete in 6.02 seconds
70.31% complete
74 / 76 text retrieved
Batch of data of row range 180-360 complete in 3.76 seconds
100% complete


In [27]:
output_path= 'unprocessed_' + csv_name
if 'df' not in locals() or df is None or 'content' not in df:
    if(os.path.exists(output_path)):
        df = pd.read_csv(output_path, on_bad_lines='skip', converters={'content': pd.eval})



In [28]:
df[['country', 'news_outlet', 'title', 'date', 'text']] = pd.DataFrame(df.content.tolist(), index= df.index)     
df

,Unnamed: 0.1,Unnamed: 0,link,content,country,news_outlet,title,date,text
0,0,0,https://www.diariodexalapa.com.mx/gossip/diego-del-rio-el-encargado-de-preparar-el-homenaje-a-silvia-pinal-en-bellas-artes-8634032.html,"[mexico, diariodexalapa, Diego del Río, el encargado de preparar el homenaje a Silvia Pinal en Bellas Artes, , Diego del Río dirige el homenaje que el 29 de agosto se le rendirá en Bellas Artes a la actriz Silvia Pinal. “Va a recorrer su legado actoral y habrá un momento que creo que va a distinguir el homenaje de otros que hace el Palacio de Bellas Artes, con el gran amor de Silvia Pinal”, adelanta el director, quien aclara que pronto darán a conocer más detalles del tributo a la artista, quien recientemente tuvo un breve retorno al teatro en una comedia musical.\n\nEste ha sido un año de...",mexico,diariodexalapa,"Diego del Río, el encargado de preparar el homenaje a Silvia Pinal en Bellas Artes",,"Diego del Río dirige el homenaje que el 29 de agosto se le rendirá en Bellas Artes a la actriz Silvia Pinal. “Va a recorrer su legado actoral y habrá un momento que creo que va a distinguir el homenaje de otros que hace el Palacio de Bellas Artes, con el gran amor de Silvia Pinal”, adelanta el director, quien aclara que pronto darán a conocer más detalles del tributo a la artista, quien recientemente tuvo un breve retorno al teatro en una comedia musical.\n\nEste ha sido un año de “primeras veces” para el director de obras como Espejos, Masterclass y El principio de Arquímedes. Debutó en c..."
1,1,1,https://www.diariodexalapa.com.mx/policiaca/asesinan-a-joven-de-tres-disparos...-y-no-hay-detenidos-todo-sobre-el-caso-8633886.html,"[mexico, diariodexalapa, Asesinan a joven de tres disparos... y no hay detenidos; todo sobre el caso, , TLAPACOYAN, Ver.- Un joven fue asesinado a balazos en la zona denominada como Oro Verde, en la salida a Teziutlán, Puebla, en esta ciudad. Los primeros informes señalan que los sicarios dispararon desde un vehículo negro en contra de la víctima, identificada como Ignacio.\n\nVecinos se percataron de que se había dado una agresión armada en contra de una persona, quien recibió varios impactos de bala sobre su humanidad.\n\nTe puede interesar: Asaltan a mujer afuera de un banco... pero los...",mexico,diariodexalapa,Asesinan a joven de tres disparos... y no hay detenidos; todo sobre el caso,,"TLAPACOYAN, Ver.- Un joven fue asesinado a balazos en la zona denominada como Oro Verde, en la salida a Teziutlán, Puebla, en esta ciudad. Los primeros informes señalan que los sicarios dispararon desde un vehículo negro en contra de la víctima, identificada como Ignacio.\n\nVecinos se percataron de que se había dado una agresión armada en contra de una persona, quien recibió varios impactos de bala sobre su humanidad.\n\nTe puede interesar: Asaltan a mujer afuera de un banco... pero los detiene la policía; los detalles\n\nPoliciaca Transportaban chapopote... pero sin papeles legales, ¿en ..."
2,2,2,https://www.diariodexalapa.com.mx/policiaca/vuelca-camioneta-en-carretera-hacia-perote-que-transportaba-8613069.html,"[mexico, diariodexalapa, Vuelca camioneta en carretera hacia Perote; ¿qué transportaba?, , Una camioneta volcó aparentemente a causa de la velocidad inmoderada, en la carretera a Perote, a la altura de Plan de Sedeño, del municipio de Las Vigas; una persona resultó lesionada, de acuerdo con rescatistas.\n\nEl accidente ocurrió antes del anochecer, cuando el chofer perdió el control de la unidad, que volcó sobre su costado derecho.\n\nTe puede interesar: ¿Lo has visto? Familia xalapeña busca a joven Salvador Durán\n\nElementos de la Policía Municipal de Las Vigas tomaron conocimiento de lo ...",mexico,diariodexalapa,Vuelca camioneta en carretera hacia Perote; ¿qué transportaba?,,"Una camioneta volcó aparentemente a causa de la velocidad inmoderada, en la carretera a Perote, a la altura de Plan de Sedeño, del municipio de Las Vigas; una persona resultó lesionada, de acuerdo con rescatistas.\n\nEl

In [29]:
print(len(df))
df = df[df['text'] != '']
print(len(df))
df

256
250


,Unnamed: 0.1,Unnamed: 0,link,content,country,news_outlet,title,date,text
0,0,0,https://www.diariodexalapa.com.mx/gossip/diego-del-rio-el-encargado-de-preparar-el-homenaje-a-silvia-pinal-en-bellas-artes-8634032.html,"[mexico, diariodexalapa, Diego del Río, el encargado de preparar el homenaje a Silvia Pinal en Bellas Artes, , Diego del Río dirige el homenaje que el 29 de agosto se le rendirá en Bellas Artes a la actriz Silvia Pinal. “Va a recorrer su legado actoral y habrá un momento que creo que va a distinguir el homenaje de otros que hace el Palacio de Bellas Artes, con el gran amor de Silvia Pinal”, adelanta el director, quien aclara que pronto darán a conocer más detalles del tributo a la artista, quien recientemente tuvo un breve retorno al teatro en una comedia musical.\n\nEste ha sido un año de...",mexico,diariodexalapa,"Diego del Río, el encargado de preparar el homenaje a Silvia Pinal en Bellas Artes",,"Diego del Río dirige el homenaje que el 29 de agosto se le rendirá en Bellas Artes a la actriz Silvia Pinal. “Va a recorrer su legado actoral y habrá un momento que creo que va a distinguir el homenaje de otros que hace el Palacio de Bellas Artes, con el gran amor de Silvia Pinal”, adelanta el director, quien aclara que pronto darán a conocer más detalles del tributo a la artista, quien recientemente tuvo un breve retorno al teatro en una comedia musical.\n\nEste ha sido un año de “primeras veces” para el director de obras como Espejos, Masterclass y El principio de Arquímedes. Debutó en c..."
1,1,1,https://www.diariodexalapa.com.mx/policiaca/asesinan-a-joven-de-tres-disparos...-y-no-hay-detenidos-todo-sobre-el-caso-8633886.html,"[mexico, diariodexalapa, Asesinan a joven de tres disparos... y no hay detenidos; todo sobre el caso, , TLAPACOYAN, Ver.- Un joven fue asesinado a balazos en la zona denominada como Oro Verde, en la salida a Teziutlán, Puebla, en esta ciudad. Los primeros informes señalan que los sicarios dispararon desde un vehículo negro en contra de la víctima, identificada como Ignacio.\n\nVecinos se percataron de que se había dado una agresión armada en contra de una persona, quien recibió varios impactos de bala sobre su humanidad.\n\nTe puede interesar: Asaltan a mujer afuera de un banco... pero los...",mexico,diariodexalapa,Asesinan a joven de tres disparos... y no hay detenidos; todo sobre el caso,,"TLAPACOYAN, Ver.- Un joven fue asesinado a balazos en la zona denominada como Oro Verde, en la salida a Teziutlán, Puebla, en esta ciudad. Los primeros informes señalan que los sicarios dispararon desde un vehículo negro en contra de la víctima, identificada como Ignacio.\n\nVecinos se percataron de que se había dado una agresión armada en contra de una persona, quien recibió varios impactos de bala sobre su humanidad.\n\nTe puede interesar: Asaltan a mujer afuera de un banco... pero los detiene la policía; los detalles\n\nPoliciaca Transportaban chapopote... pero sin papeles legales, ¿en ..."
2,2,2,https://www.diariodexalapa.com.mx/policiaca/vuelca-camioneta-en-carretera-hacia-perote-que-transportaba-8613069.html,"[mexico, diariodexalapa, Vuelca camioneta en carretera hacia Perote; ¿qué transportaba?, , Una camioneta volcó aparentemente a causa de la velocidad inmoderada, en la carretera a Perote, a la altura de Plan de Sedeño, del municipio de Las Vigas; una persona resultó lesionada, de acuerdo con rescatistas.\n\nEl accidente ocurrió antes del anochecer, cuando el chofer perdió el control de la unidad, que volcó sobre su costado derecho.\n\nTe puede interesar: ¿Lo has visto? Familia xalapeña busca a joven Salvador Durán\n\nElementos de la Policía Municipal de Las Vigas tomaron conocimiento de lo ...",mexico,diariodexalapa,Vuelca camioneta en carretera hacia Perote; ¿qué transportaba?,,"Una camioneta volcó aparentemente a causa de la velocidad inmoderada, en la carretera a Perote, a la altura de Plan de Sedeño, del municipio de Las Vigas; una persona resultó lesionada, de acuerdo con rescatistas.\n\nEl

In [30]:
if 'page' in df:
  df = df.drop(['page'], axis=1)
if 'content' in df:
  df = df.drop(['content'], axis=1)
if 'Unnamed: 0' in df:
    df = df.drop(['Unnamed: 0'], axis=1)
if 'Unnamed: 0.1' in df:
    df = df.drop(['Unnamed: 0.1'], axis=1)

In [31]:
df

,link,country,news_outlet,title,date,text
0,https://www.diariodexalapa.com.mx/gossip/diego-del-rio-el-encargado-de-preparar-el-homenaje-a-silvia-pinal-en-bellas-artes-8634032.html,mexico,diariodexalapa,"Diego del Río, el encargado de preparar el homenaje a Silvia Pinal en Bellas Artes",,"Diego del Río dirige el homenaje que el 29 de agosto se le rendirá en Bellas Artes a la actriz Silvia Pinal. “Va a recorrer su legado actoral y habrá un momento que creo que va a distinguir el homenaje de otros que hace el Palacio de Bellas Artes, con el gran amor de Silvia Pinal”, adelanta el director, quien aclara que pronto darán a conocer más detalles del tributo a la artista, quien recientemente tuvo un breve retorno al teatro en una comedia musical.\n\nEste ha sido un año de “primeras veces” para el director de obras como Espejos, Masterclass y El principio de Arquímedes. Debutó en c..."
1,https://www.diariodexalapa.com.mx/policiaca/asesinan-a-joven-de-tres-disparos...-y-no-hay-detenidos-todo-sobre-el-caso-8633886.html,mexico,diariodexalapa,Asesinan a joven de tres disparos... y no hay detenidos; todo sobre el caso,,"TLAPACOYAN, Ver.- Un joven fue asesinado a balazos en la zona denominada como Oro Verde, en la salida a Teziutlán, Puebla, en esta ciudad. Los primeros informes señalan que los sicarios dispararon desde un vehículo negro en contra de la víctima, identificada como Ignacio.\n\nVecinos se percataron de que se había dado una agresión armada en contra de una persona, quien recibió varios impactos de bala sobre su humanidad.\n\nTe puede interesar: Asaltan a mujer afuera de un banco... pero los detiene la policía; los detalles\n\nPoliciaca Transportaban chapopote... pero sin papeles legales, ¿en ..."
2,https://www.diariodexalapa.com.mx/policiaca/vuelca-camioneta-en-carretera-hacia-perote-que-transportaba-8613069.html,mexico,diariodexalapa,Vuelca camioneta en carretera hacia Perote; ¿qué transportaba?,,"Una camioneta volcó aparentemente a causa de la velocidad inmoderada, en la carretera a Perote, a la altura de Plan de Sedeño, del municipio de Las Vigas; una persona resultó lesionada, de acuerdo con rescatistas.\n\nEl accidente ocurrió antes del anochecer, cuando el chofer perdió el control de la unidad, que volcó sobre su costado derecho.\n\nTe puede interesar: ¿Lo has visto? Familia xalapeña busca a joven Salvador Durán\n\nElementos de la Policía Municipal de Las Vigas tomaron conocimiento de lo sucedido y se trasladaron al sitio del percance para ayudar.\n\nPoliciaca Se incendia vivie..."
3,https://www.diariodexalapa.com.mx/policiaca/secuestradores-de-xalapa-y-coatepec-pasaran-50-anos-de-prision-cuando-ocurrieron-los-delitos-8610004.html,mexico,diariodexalapa,Sentencian a 50 años de prisión a secuestradores; operaban en Xalapa y Coatepec,,"Dos secuestradores que operaban en Xalapa y Coatepec fueron sentenciados a una pena de 50 años en el penal de Pacho Viejo, de acuerdo con autoridades ministeriales.\n\nLos delincuentes son Leonardo y Lorenzo, quienes entre el 23 y 25 de marzo del 2020 secuestraron a dos personas y las mantuvieron encerradas en una casa de seguridad de la colonia Obreros Textiles, de Xalapa, según el proceso penal 03/2022.\n\n¿Cómo ocurrieron los secuestros?\n\nEn esa ocasión, la Unidad Especializada en Combate al Secuestro (UECS) realizó las primeras investigaciones y con base en un operativo descubrió a l..."
4,https://www.diariodexalapa.com.mx/policiaca/presunto-policia-de-la-ssp-extorsionaba-a-duenos-de-bares-en-xalapa-todo-sobre-el-caso-8625181.html,mexico,diariodexalapa,Se hacía pasar como policía de la SSP... extorsionaba a dueños de bares en Xalapa,,"La Fiscalía General del Estado (FGE) inició una carpeta de investigación en contra de un presunto extorsionador que supuestamente se hizo pasar como elemento de Seguridad Pública para extorsionar a dueños de bares en esta ciudad.\n\nDe acuerdo con autoridades de la SSP, el presunto responsable se encuentra identificado y cometió el robo de identidad para ha

In [32]:
df.to_csv(csv_name)

In [33]:
if(os.path.exists(output_path)):
    os.remove(output_path)
    print(('unprocessed file deleted'))

unprocessed file deleted
